In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

import warnings
warnings.simplefilter('ignore')

## Billionaires

In [ ]:
billionaires = Table.read_table('data/billionaires.csv')
billionaires

In [ ]:
(billionaires.group('citizenship') # group by citizenship 
             .sort(1, descending=True) # or .sort('count',...) # sort by COUNT
             .barh('citizenship')) # plot a bar chart, with citizenship on y-axis

In [ ]:
(billionaires.group('industry')
             .sort(1, descending=True) # make it easier to read, sort by greatest to smallest
             .barh('industry'))

In [ ]:
# cross-clasify to compare billionaires in different industries
# and citizenships
billionaires.pivot('citizenship', 'industry').show()

In [ ]:
billionaires.pivot('industry','citizenship').show()
# first argument - column headings
# second argument - row headings
# values inside pivot table are count of billionaries that belong to said industry and hold citizenship
# in given country

In [ ]:
countries = Table.read_table('data/countries.csv')
countries.where('Continent', 'Oceania').show()

In [ ]:
countries.where('Continent', 'North America').show()

In [ ]:
billionaires = billionaires.join('citizenship', countries, 'Country')
billionaires
# the column in our first argument 'citizenship', becomes the first column
# the last column(s) is now the remainder column(s) in the third argument, countries (second table)

In [ ]:
billionaires.pivot('Continent', 'industry').show()

In [ ]:
ceos= Table.read_table('data/ceo.csv').relabel('amount', 'amount $1000')
ceos
#ceos.sort('amount')
# party 100 = democratic party
# part 200 = republican

In [ ]:
ceos.hist('amount $1000')

In [ ]:
ceos.sort('amount $1000', descending=True)

In [ ]:
ceos.sort('amount $1000', descending=False)

In [ ]:
ceos.hist('amount $1000', bins=np.arange(0, 1e5, 1e3), unit='$1000')

In [ ]:
billionaires.where('name','Gil Omenn')

In [ ]:
ceos.where('ceo', 'Gil Omenn')

In [ ]:
b_donations = billionaires.join('name', ceos, 'ceo')
b_donations.sort('amount $1000', descending = False)
b_donations
b_donations.where('name','Gil Omenn') 
# Gil Omenn was not a billionaire
# so he was not included in b_donations table

# b_donations are billionaires who are ceos too

In [ ]:
# how much of their wealth are they donating to political parties?
# compare the proportion of their wealth they gave away
b_donations.where('amount $1000', are.below(6e6)).scatter('net worth', 'amount $1000')

# how would you describe the relationship between billionaire ceos' net worth
# and the political donations they made, based on the graph below?
# 1e6 (in $1000) = $1,000,000,000
# 1e11 = $100,000,000,000

In [ ]:
donation_prop = ((b_donations.column('amount $1000')*1000) / b_donations.column('net worth'))
b_donations = b_donations.with_column('Contribution%', donation_prop)
b_donations = b_donations.sort('amount $1000', descending=True)
b_donations.set_format('Contribution%', PFormatter)
b_donations.sort('Contribution%', descending = True)

In [ ]:
help(Table.set_format)

In [ ]:
# visualize a histogram of the contribution % 
# CHALLENGE:
# can you generate a histogram comparing contribution % ?


# BONUS: what is the average contribution %? in other words, on average
# how much of billionaire ceos' wealth did they donate to political parties?
b_donations.hist('Contribution%')
np.average(b_donations.column('Contribution%'))
# average contribution % is 1.89% of our billionaire ceos' wealth

In [ ]:
.02 / 100 * 20000

## Meteorites

In [ ]:
meteorites = Table.read_table('data/meteorites_na.csv')
meteorites.sort('year', descending = False)

In [ ]:
meteorites.group('class').sort(1, descending=True)

In [ ]:
meteorites.group('how')

In [ ]:
meteorites.group('how', np.average)

In [ ]:
meteorites.hist('year', bins=np.arange(1800, 2020, 5), group='how')
# what is the difference between these two groups? fell vs found?


In [ ]:
meteorites.scatter('year', 'mass')
#10,000,000

In [ ]:
meteorites.where('mass', are.below(1e5)).where('year', are.between(1950,2021)).scatter('year', 'mass')

In [ ]:
new_fell = meteorites.where('how', 'Fell').where('year', are.above(1950))
new_found  = meteorites.where('how', 'Found').where('year', are.above(1950))
sum(new_found.group('class').sort('count', descending = True).column('count'))

In [ ]:
# group meteorites that fell after 1950 by class
new_fell_grouped_class = new_fell.group('class')
# sorting by greatest to smallest count by class
new_fell_grouped_class = new_fell_grouped_class.sort('count', descending = True)
# extracting count column as an array
new_fell_grouped_class_array = new_fell_grouped_class.column('count')
# so we can find total number of meteorites that fell, after 1950
sum(new_fell_grouped_class_array)

## Maps

In [ ]:
Marker.map_table(new_fell.select('lat', 'lon', 'name'))

In [ ]:
Circle.map_table(new_found.select('lat', 'lon', 'name'), radius=10)

## Bikes

If you loaded the meteorites dataset above, now might be a good time to restart your kernel, run the cell at the top of the notebook, then come back here. 

The bike datasets in this section is big, and could lead to kernel crashes.

In [ ]:
trip = Table.read_table('data/trip.csv')
trip

In [ ]:
1800/60

In [ ]:
commute = trip.where('Duration', are.below(1800))
commute.hist('Duration')

In [ ]:
commute.hist('Duration', bins=60, unit='second')

In [ ]:
# Percent of people who have a ride duration between 500 and 250 seconds
(500-250) * 0.15 

In [ ]:
starts = commute.group('Start Station').sort('count', descending=True)
starts

In [ ]:
commute.pivot('Start Station', 'End Station')

In [ ]:
duration = trip.select('Start Station', 'End Station', 'Duration')
duration

In [ ]:
shortest = duration.group(['Start Station', 'End Station'], min).relabeled('Duration min', 'Minimum Duration')
shortest

In [ ]:
from_cc = shortest.where('Start Station', are.containing('Civic Center BART')).sort('Minimum Duration')
from_cc

## Maps, again

In [ ]:
stations = Table.read_table('data/station.csv')
stations

In [ ]:
Marker.map_table(stations.select('lat', 'long', 'name'))

In [ ]:
sf = stations.where('landmark', 'San Francisco')
Circle.map_table(sf.select('lat', 'long', 'name'), color='green', radius=15)

## Extra practice

### Write a Python expression below each of the following descriptions that computes its value. You may use more than one line.

In [ ]:
# The name of the station where the most rentals ended (assume no ties).


In [ ]:
# The number of stations for which the average duration ending at that station was more than 10 minutes.


In [ ]:
# The number of stations that have more than 500 starts AND more than 500 ends

In [ ]:
# The name of the station where the most rentals ended (assume no ties).
# First, find end counts
# Then, find the station with the highest end count
trip.group('End').sort('count', descending=True).column(0).item(0)

In [ ]:
# The number of stations for which the average duration ending at that station was more than 10 minutes.

# First, find the average end time for each station
# Then, keep the ones above 10 minutes
# Then, count them
trip.group('End', np.average).where(2, are.above(10*60)).num_rows

In [ ]:
# The number of stations that have more than 500 starts AND more than 500 ends.

# First, group the trip on starts and ends, separately
# Then, join the two grouped tables
# Then, count the number of rows where the stations' start and end counts are bove 500
start_counts, end_counts = trip.group('Start').relabeled('count', 'Start Count'), trip.group('End').relabeled('count', 'End Count')
start_counts.join('Start', end_counts, 'End').where(
    'Start Count', are.above(500)).where('End Count', are.above(500)).num_rows